In [1]:
# Libaries

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score

import statsmodels.api as sm
from statsmodels.formula.api import ols

C:\Users\ljant\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\ljant\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:62: UserWarning: Pandas requires version '1.3.4' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


# Train Data Preparation

In [2]:
#Get the data and prepare df
df_train = pd.read_csv(r'C:\Users\ljant\Desktop\Ironhack\Projects\01_Project\FIFA_TRAIN_DATA.CSV', sep='?')
def header_function (data):
    cols = []
    for i in range (len(data.columns)):
        cols.append(data.columns[i].lower().replace(' ','_'))
    
    data.columns = cols

    return data

header_function(df_train)

# Changing the columns with values of 'xx + x' into the result of this 'euqation',replacing them, and using the mean of all these variables
def calculation_columns (data, columns):
    for col in columns:
        df_train[col] = df_train[col].str.split('+').apply(lambda x: sum(map(int, x))) 

fix_columns = ['ls', 'st', 'rs', 'lw', 'lf', 'cf', 'rf', 'rw', 'lam',
               'cam', 'ram', 'lm', 'lcm', 'cm', 'rcm', 'rm', 'lwb',
               'ldm', 'cdm', 'rdm', 'rwb', 'lb', 'lcb', 'cb', 'rcb',
               'rb', 'gk']

calculation_columns(df_train, fix_columns)
df_train['skill_mean'] = df_train[['ls','st','rs', 'lw', 'lf','cf', 'rf','rw', 'lam', 'cam', 'ram', 'lm', 'lcm', 'cm', 'rcm',
                                   'rm','pac','sho','pas','dri','def','phy', 'lwb', 'ldm', 'cdm', 'rdm', 'rwb', 'lb', 'lcb',
                                   'cb', 'rcb','rb']].mean(axis = 1)

#Getting the columns we need with only the top 1.500 players --> to train the model
df_train = df_train[['ova', 'age', 'reactions', 'balance', 'jumping','stamina', 'strength','composure', 'skill_mean', 'hits',
                     'movement', 'power', 'skill', 'crossing', 'finishing','curve', 'fk_accuracy','long_passing',
                     'ball_control', 'acceleration', 'sprint_speed', 'agility','shot_power','aggression', 'positioning',
                     'vision','penalties','marking','standing_tackle','sliding_tackle']]

df_train = df_train.sort_values(by='ova', ascending=False).head(1500)

#Further cleaning of the data:
df_train['hits'] = df_train['hits'].str.replace('K', '').astype(float)
df_train['hits'] *= 1000
mean_values = df_train.mean()
df_train = df_train.fillna(mean_values)

df_train

,ova,age,reactions,balance,jumping,stamina,strength,composure,skill_mean,hits,...,sprint_speed,agility,shot_power,aggression,positioning,vision,penalties,marking,standing_tackle,sliding_tackle
602,93,33,94,95.0,68.0,72,69,96.0,80.53125,830000.0,...,80,91.0,86,44,93.0,95.0,75,32,35,24.0
111,92,35,95,71.0,95.0,84,78,95.0,78.90625,750000.0,...,91,87.0,94,63,95.0,82.0,84,28,32,24.0
1845,91,31,93,82.0,84.0,76,86,88.0,78.06250,487000.0,...,78,77.0,89,81,94.0,79.0,88,35,42,19.0
2946,91,27,88,49.0,78.0,41,78,68.0,45.40625,220000.0,...,60,67.0,59,34,11.0,65.0,11,27,12,18.0
2138,91,28,91,83.0,62.0,81,50,93.0,77.90625,546000.0,...,89,96.0,80,51,87.0,90.0,92,35,30,29.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3961,75,26,72,81.0,52.0,55,47,73.0,68.09375,7000.0,...,55,82.0,76,45,72.0,78.0,59,55,57,55.0
3833,75,25,69,77.0,38.0,68,60,71.0,68.81250,82000.0,...,75,76.0,74,55,69.0,74.0,55,65,59,53.0
3925,75,33,74,32.0,56.0,42,72,68.0,38.56250,10000.0,...,48,31.0,56,20,19.0,65.0,20,18,12,18.0
3843,75,26,74,37.0,75.0,72,86,67.0,59.96875,173000.0,...,51,49.0,53,80,34.0,37.0,31,76,74,70.0


# Test Data Preparation

In [3]:
#Get the data and prepare df
df_test = pd.read_csv(r'C:\Users\ljant\Desktop\Ironhack\Projects\01_Project\FIFA_TEST_DATA.CSV', sep='?')
def header_function (data):
    cols = []
    for i in range (len(data.columns)):
        cols.append(data.columns[i].lower().replace(' ','_'))
    
    data.columns = cols

    return data

header_function(df_test)

# Changing the columns with values of 'xx + x' into the result of this 'euqation',replacing them, and using the mean of all these variables
def calculation_columns (data, columns):
    for col in columns:
        df_test[col] = df_test[col].str.split('+').apply(lambda x: sum(map(int, x)))

fix_columns = ['ls', 'st', 'rs', 'lw', 'lf', 'cf', 'rf', 'rw', 'lam',
               'cam', 'ram', 'lm', 'lcm', 'cm', 'rcm', 'rm', 'lwb',
               'ldm', 'cdm', 'rdm', 'rwb', 'lb', 'lcb', 'cb', 'rcb',
               'rb', 'gk']

calculation_columns(df_test, fix_columns)
df_test['skill_mean'] = df_test[['ls','st','rs', 'lw', 'lf','cf', 'rf','rw', 'lam', 'cam', 'ram', 'lm', 'lcm', 'cm', 'rcm',
                                 'rm','pac','sho','pas','dri','def','phy', 'lwb', 'ldm', 'cdm', 'rdm', 'rwb', 'lb', 'lcb',
                                 'cb', 'rcb','rb']].mean(axis = 1)
# Save the names of the df:
names_test = df_test['name']

# Save values of the players:
value_test = df_test ['value']

# Getting the columns we need
df_test = df_test[['age', 'reactions', 'balance', 'jumping','stamina', 'strength','composure', 'skill_mean', 'hits',
                   'movement', 'power', 'skill', 'crossing', 'finishing', 'curve', 'fk_accuracy','long_passing',
                   'ball_control', 'acceleration', 'sprint_speed', 'agility','shot_power','aggression', 'positioning',
                   'vision','penalties', 'marking','standing_tackle','sliding_tackle']]

#Further cleaning of the data:
df_test['hits'] = df_test['hits'].str.replace('K', '').astype(float)
df_test['hits'] *= 1000
mean_values = df_test.mean()
df_test = df_test.fillna(mean_values)

df_test

,age,reactions,balance,jumping,stamina,strength,composure,skill_mean,hits,movement,...,sprint_speed,agility,shot_power,aggression,positioning,vision,penalties,marking,standing_tackle,sliding_tackle
0,19,49,45.0,62.0,44,80,45.0,46.71875,4000.0,246,...,55,49.0,58,44,66.0,41.0,55,18,20,15.0
1,21,62,75.0,52.0,59,47,58.0,61.25000,33000.0,353,...,68,77.0,58,58,56.0,61.0,59,55,54,51.0
2,21,56,63.0,56.0,61,69,52.0,57.21875,4000.0,326,...,76,62.0,40,53,51.0,49.0,41,55,60,61.0
3,29,55,60.0,66.0,66,78,65.0,53.65625,4000.0,305,...,62,63.0,56,67,22.0,35.0,24,72,71,75.0
4,30,75,60.0,71.0,79,76,75.0,72.28125,17000.0,309,...,54,64.0,77,69,56.0,81.0,55,79,78,81.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3420,27,65,86.0,65.0,69,56,68.0,59.12500,3000.0,344,...,67,55.0,55,51,42.0,41.0,40,72,74,70.0
3421,26,70,77.0,81.0,79,61,63.0,63.90625,12000.0,381,...,79,80.0,66,39,55.0,45.0,52,64,67,63.0
3422,21,55,66.0,65.0,60,80,54.0,55.25000,4000.0,359,...,80,74.0,70,75,66.0,57.0,68,21,21,14.0
3423,26,65,66.0,73.0,86,63,61.0,64.43750,13000.0,349,...,74,71.0,44,75,41.0,55.0,38,68,70,71.0


# Regression Model

In [4]:
# preparing dataframes

# Define target
target_train = df_train[['ova']].reset_index(drop = True)

# Drop traget
df_model_train = df_train.drop(['ova'], axis = 1).reset_index(drop = True)

# Multiple regression - train data

Y = target_train
X = df_model_train

X = sm.add_constant(X)

# The model:
model = sm.OLS(Y, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    ova   R-squared:                       0.709
Model:                            OLS   Adj. R-squared:                  0.703
Method:                 Least Squares   F-statistic:                     123.5
Date:                Sat, 28 Oct 2023   Prob (F-statistic):               0.00
Time:                        10:05:29   Log-Likelihood:                -2870.7
No. Observations:                1500   AIC:                             5801.
Df Residuals:                    1470   BIC:                             5961.
Df Model:                          29                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              33.2190      1.015     32.736      0.000      31.228      35.210
age                 0.0688      0.014      4.844      0.000       0.041       0.097
reactions           0.1748      0.017     10.376      0.000       0.142       0.208
balance            -0.0469      0.009     -4.947      0.000      -0.065      -0.028
jumping             0.0291      0.008      3.543      0.000       0.013       0.045
stamina            -0.0086      0.009     -0.919      0.358      -0.027       0.010
strength            0.0200      0.009      2.153      0.031       0.002       0.038
composure           0.0572      0.008      6.918      0.000       0.041       0.073
skill_mean          1.1295      0.061     18.466      0.000       1.010       1.249
hits             3.234e-06   3.55e-07      9.107      0.000    2.54e-06    3.93e-06
movement            0.0511      0.007      7.084      0.000       0.037       0.065
power              -0.0350      0.007     -4.787      0.000      -0.049      -0.021
skill              -0.1083      0.011     -9.620      0.000      -0.130      -0.086
crossing           -0.0262      0.007     -3.913      0.000      -0.039      -0.013
finishing          -0.0611      0.008     -8.043      0.000      -0.076      -0.046
curve               0.1104      0.013      8.241      0.000       0.084       0.137
fk_accuracy         0.1137      0.012      9.514      0.000       0.090       0.137
long_passing        0.0634      0.013      5.041      0.000       0.039       0.088
ball_control        0.0351      0.020      1.774      0.076      -0.004       0.074
acceleration       -0.0856      0.012     -6.984      0.000      -0.110      -0.062
sprint_speed       -0.0672      0.012     -5.630      0.000      -0.091      -0.044
agility            -0.0564      0.010     -5.421      0.000      -0.077      -0.036
shot_power          0.0223      0.013      1.742      0.082      -0.003       0.047
aggression         -0.0352      0.005     -6.933      0.000      -0.045      -0.025
positioning        -0.0962      0.008    -11.995      0.000      -0.112      -0.080
vision             -0.0669      0.008     -8.278      0.000      -0.083      -0.051
penalties          -0.0045      0.005     -0.822      0.411      -0.015       0.006
marking            -0.0435      0.007     -6.645      0.000      -0.056      -0.031
standing_tackle    -0.0839      0.010     -8.820      0.000      -0.103      -0.065
sliding_tackle     -0.0614      0.008     -7.495      0.000      -0.077      -0.045
==============================================================================
Omnibus:                       55.817   Durbin-Watson:                   1.310
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               71.171
Skew:                           0.396   Prob(JB):                     3.51

In [5]:
# Predictions
df_model_test = sm.add_constant(df_test)

lm = linear_model.LinearRegression()
model = lm.fit(X,Y)
lm.score(X,Y)

predictions = lm.predict(df_model_test)

from sklearn.metrics import mean_absolute_error

predicted_ova = lm.predict(X)

print("R2 value is = ",round(r2_score(Y, predicted_ova),2))
print("The intercept of the model is = ",lm.intercept_)
print("The coefficients of the model are = ",lm.coef_)

mse = mean_squared_error(Y, predicted_ova)
mae = mean_absolute_error(Y, predicted_ova)
print("The mse of the model is = ", round(mse,2))
print("The root mse of the model is = ",round(np.sqrt(mse),2))
print("The mean absolute error of the model is = ",round(mae,2))

C:\Users\ljant\anaconda3\lib\site-packages\sklearn\utils\validation.py:571: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  array.dtypes.apply(is_sparse).any()):
C:\Users\ljant\anaconda3\lib\site-packages\sklearn\utils\validation.py:571: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  array.dtypes.apply(is_sparse).any()):


R2 value is =  0.71
The intercept of the model is =  [33.21899943]
The coefficients of the model are =  [[ 4.12625483e-14  6.88009771e-02  1.74826782e-01 -4.68807019e-02
   2.91182873e-02 -8.64380532e-03  1.99921443e-02  5.72494575e-02
   1.12951229e+00  3.23388136e-06  5.10624194e-02 -3.50101623e-02
  -1.08317015e-01 -2.61994905e-02 -6.11436032e-02  1.10351171e-01
   1.13662714e-01  6.34277851e-02  3.51108949e-02 -8.56059460e-02
  -6.71957136e-02 -5.64206401e-02  2.23334677e-02 -3.52359149e-02
  -9.61539270e-02 -6.68833487e-02 -4.45216470e-03 -4.34793121e-02
  -8.39157917e-02 -6.13680144e-02]]
The mse of the model is =  2.69
The root mse of the model is =  1.64
The mean absolute error of the model is =  1.28


C:\Users\ljant\anaconda3\lib\site-packages\sklearn\utils\validation.py:571: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  array.dtypes.apply(is_sparse).any()):
C:\Users\ljant\anaconda3\lib\site-packages\sklearn\utils\validation.py:571: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  array.dtypes.apply(is_sparse).any()):
C:\Users\ljant\anaconda3\lib\site-packages\sklearn\utils\validation.py:571: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  array.dtypes.apply(is_sparse).any()):
C:\Users\ljant\anaconda3\lib\site-packages\sklearn\utils\validation.py:571: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  array.dtypes.apply(is_sparse).any()):
C:\Users\lja

In [6]:
# Adding the new column 'predicted_ova' so the df
df_test['predicted_ova'] = lm.predict(df_model_test)

#Creating a variable with the best players
df_best_players = df_test['predicted_ova'].round().astype(int)

#Show the result of the top 10 players based on the regression model
df_best = pd.DataFrame({'Player': names_test, 'Ova': df_best_players, 'Value': value_test, 'Age': df_test['age']})
df_best = df_best.sort_values(by='Ova', ascending=False).reset_index(drop = True)
df_best.head(10)

C:\Users\ljant\anaconda3\lib\site-packages\sklearn\utils\validation.py:571: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  array.dtypes.apply(is_sparse).any()):


,Player,Ova,Value,Age
0,M. Neuer,89,€29M,34
1,V. van Dijk,87,€75.5M,28
2,S. Mané,87,€78M,28
3,S. Handanovič,86,€16M,35
4,P. Lahm,86,€29.5M,32
5,L. Suárez,86,€31.5M,33
6,N. Kanté,85,€51M,29
7,F. de Jong,85,€50.5M,23
8,F. Valverde,85,€36M,21
9,K. Mbappé,85,€105.5M,21
